In [682]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames
using Plots
using DelimitedFiles
using Random

In [683]:
path = "C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/";

In [684]:
#load social and ecological data and orgnize 
FL = repeat([Float64], inner=15)
dtype = append!([String], FL);

regional_EF = CSV.File(string(path,"EF_SC.csv"),header=1,delim=",", types=dtype) |> DataFrame    
capacity = CSV.File(string(path,"capacity30_.csv"),header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File(string(path,"distance30.csv"),header=1,delim=",") |> DataFrame 
LCA_model = CSV.File(string(path,"LCA_model.csv"),header=1,delim=",") |> DataFrame 
D_Dsoc = CSV.File(string(path,"D_Dsoc.csv"),header=1,delim=",") |> DataFrame
emi_sink = CSV.File(string(path,"emission_sink.csv"),header=1,delim=",") |> DataFrame;

In [685]:
global_sink = 1.099e10                        # global pub (ocean) CO2 sequestration (ton/yr)
global_sink_tot = 2.236e10                  # global total (ocean+land) CO2 sequestration (ton/yr)
global_emi = 3.53e10                          # global CO2 emission (ton/yr)
global_gdp = 96882e9                          # 2021 global GDP ($/yr)
es_ratio = global_sink/global_emi
es_ratio_tot = global_sink_tot/global_emi

emission_c = emi_sink[!, "emission"]          # national CO2 emission (ton/yr)
sink_c = emi_sink[!, "sink ton/yr"] .* 0.6          # national CO2 sink (ton/yr)
Dsoc = D_Dsoc[!, "Dsoc ton/yr"]           # national CO2 sink (ton/yr)

EF_trans = 1.005/10000                        # ton CO2/km*ton (The average freight truck in the U.S. emits 161.8 grams of CO2 per ton-mile)
# EF_trans = 86/1000000                          #5-LH trucks emit on average 56.6 gCO2/tkm while 5-RD emit 84.0 gCO2/tkm
process = LCA_model[!,"process"]
countries = capacity[!,"country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

# seperate model
cathode = collect(1:4)
cell = collect(5:10)
noncell = [12,13]
battery = [11,14]
scaler = LCA_model[!,"scaler"]
penalty = 5; # ton/yr;

In [686]:
# list = [1,4,5,31,37,38,49];
# list = [1,4,5,11,21,31,37,41,49];
# list = [1,3,4,5,9,17,18,31,32,37,38,41,43,44,48,49];
# list = [1];
# list = [1, 4, 11, 21, 31, 37, 41, 49];
list = [1,4,5,31,37,41,49]

7-element Vector{Int64}:
  1
  4
  5
 31
 37
 41
 49

In [687]:
scen_eff = CSV.File(string(path,"Scenarios/Scen_Efficiency2.csv"), header=1, delim=",") |> DataFrame
scen_eff = Matrix(scen_eff)[:, 2:end]
scen_eff = scen_eff[:, list]
num_omega = size(scen_eff)[2]

scen_dmd = CSV.File(string(path,"Scenarios/Scen_Demand.csv"), header=1, delim=",") |> DataFrame
scen_dmd = Matrix(scen_dmd)[2:end][list]
# scen_dmd = ones(num_omega) .* 452000

Omega = ones(Float64, ncty, num_omega, nproc);  # Omega[ncty,nscena,nproc]
Omega[:, :, 1] = scen_eff;

dmd_fixed = 452000

452000

---

### Solve Subproblem - Get Dual $\pi$ & $\alpha$

In [688]:
function subprob(x_hat, s) # s represents specific scenario     
    
    model = Model(Gurobi.Optimizer)
    set_silent(model)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    @variable(model, unmeet >= 0)

    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == Omega[i,s,k] * x_hat[i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,5] * scaler[k]) for k in cathode for j in 1:ncty]
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,11] * scaler[k]) for k in cell for j in 1:ncty]
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,14] * scaler[k]) for k in noncell for j in 1:ncty]
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,15] * scaler[k]) for k in battery for j in 1:ncty] 
    
    cstr_fmk = [@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) + unmeet == scen_dmd[s])]
    
    
    transD = Vector{AffExpr}(undef, ncty)
    for j in 1:ncty
        arc_emi = 0
        for i in 1:ncty
            amount = sum(y[i,j,k] for k in 1:nproc)
            arc_emi += amount * distance[!, 2:end][i,j] * EF_trans
        end
        transD[j] = arc_emi  # ncty*1 matrix
    end
    
    
    pro_sink = zeros(AffExpr, ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = (sum(y[i,j,k] for j in 1:ncty)) * regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = sum(pro_sink)
    
    
    sink_ratio = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            sink_ratio[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    overprod_penalty = sum((x_hat - sum(y, dims=2)[:,1,:]) .* sink_ratio)
    
    
    ############### Obj function ##############
    @objective(model, Min, sum(transD) + unmeet * penalty + overprod_penalty)
    JuMP.optimize!(model)
    
    qy_hat = JuMP.objective_value(model)
    sub_y = JuMP.value.(y)
    sub_unmeet = JuMP.value.(unmeet)
    
    
    ################## get dual and calculate cuts ################
    dual_pi = zeros(ncty,nproc)

    op_pi = [getdual(con) for con in cstr_op]
    cth_pi = [getdual(con) for con in cstr_cth]
    cell_pi = [getdual(con) for con in cstr_cell]
    noncell_pi = [getdual(con) for con in cstr_noncell]
    battery_pi = [getdual(con) for con in cstr_battery]
    fmk_alp = [getdual(con) for con in cstr_fmk]


    dual_pi[:, cathode] = reshape(cth_pi, ncty, length(cathode)) .* Omega[:,s,cathode] .* repeat(scaler[cathode]', ncty, 1)
    dual_pi[:, cell] = reshape(cell_pi, ncty, length(cell)) .* Omega[:,s,cell] .* repeat(scaler[cell]', ncty, 1)
    dual_pi[:, noncell] = reshape(noncell_pi, ncty, length(noncell)) .* Omega[:,s,noncell] .* repeat(scaler[noncell]', ncty, 1)
    dual_pi[:, battery] = reshape(battery_pi, ncty, length(battery)) .* Omega[:,s,battery] .* repeat(scaler[battery]', ncty, 1)
    dual_pi += reshape(op_pi, ncty, nproc) .* Omega[:,s,:]
    
    dual_alpha = sum(fmk_alp) * scen_dmd[s]

    #######################
    result = Dict(["qyhat"=>qy_hat, "y_opt"=>sub_y, "unmeet_opt"=>sub_unmeet, "pi"=>dual_pi, "alp"=>dual_alpha]);
    return result
    
end

subprob (generic function with 1 method)

In [689]:
function masterprob(cuts)
    model = Model(Gurobi.Optimizer)
    set_silent(model)
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, theta >= M)
    
    for k in 1:nproc
        for i in 1:ncty
            @constraint(model, x[i,k] <= 0.8*capacity[!, 2:end][i,k])
        end
    end

    x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
    x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
    x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
    x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];
    

    ###########
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1) 
    
    G = 0
    g = 0
    if cuts != []
        for cut in cuts
            G = cut["gradient"]
            g = cut["intersection"]
            @constraint(model, theta >= sum(x .* G) + g); 
        end
    end
    
    
    @objective(model, Min, sum(proD - proS) + theta)
    JuMP.optimize!(model);
    
    x_hat = JuMP.value.(x)
    theta_hat = JuMP.value.(theta)
    z_lb = JuMP.objective_value(model)
    
    result = Dict(["x_hat"=>x_hat, "z_lb"=>z_lb])
    return result

end

masterprob (generic function with 1 method)

In [690]:
function add_cuts(x)
    G = [] # gradient
    g = [] # intersection
    qy = 0
    
    
    for s in 1:num_omega
        res = subprob(x, s)        
        push!(G, res["pi"])   # gradient
        push!(g, res["alp"])  # intersection
        qy += res["qyhat"]
    end

    
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1) 
    
    
    z_hat = sum(proD - proS) + qy/num_omega
    Exp_G = sum(G) / num_omega
    Exp_g = sum(g) / num_omega
    
    cut = Dict(["gradient"=>Exp_G, "intersection"=>Exp_g])
    return cut, z_hat

end

add_cuts (generic function with 1 method)

In [691]:
function main(toler)
    i = 0
    z_ub = 1e8
    x_opt = 0
    
    
    cuts = []
    res0 = masterprob(cuts)
    x_hat = res0["x_hat"]
    z_lb = res0["z_lb"]
    
    
    while (z_ub - z_lb) > toler * min(abs(z_ub), abs(z_lb))
        new_cut, z_hat = add_cuts(x_hat)
        
        if z_hat < z_ub
            x_opt = x_hat
            z_ub = z_hat
        end
        
        push!(cuts, new_cut)
        
        res1 = masterprob(cuts)
        z_lb = res1["z_lb"]
        x_hat = res1["x_hat"]
        
        i += 1
    end
    
    
    result = Dict(["z_lb"=>z_lb, "z_ub"=>z_ub, "x_opt"=>x_opt, "iteration"=>i])
    return result
    
end

main (generic function with 1 method)

---

In [692]:
M = 0

0

In [695]:
z_ub = 1e8
x_opt = 0


cuts = []
res0 = masterprob(cuts)
x_hat = res0["x_hat"]
z_lb = res0["z_lb"]


for i in 1:5
    new_cut, z_hat = add_cuts(x_hat)

    if z_hat < z_ub
        x_opt = x_hat
        z_ub = z_hat
    end

    push!(cuts, new_cut)

    res1 = masterprob(cuts)
    z_lb = res1["z_lb"]
    x_hat = res1["x_hat"]
    println(-z_lb+z_ub)

end

Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
2.7874744341209135e6
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-

-----

In [643]:
M = 0
toler = 0.01

0.01

In [644]:
i = 0
z_ub = 1e8
x_opt = 0

cuts = []
res0 = masterprob(cuts)
x_hat = res0["x_hat"]
z_lb = res0["z_lb"]

Academic license - for non-commercial use only - expires 2024-12-26


-3.3870410868231794e6

In [645]:
xhat11 = x_hat

30×15 Matrix{Float64}:
  4960.0       0.0  0.0           0.0  …     0.0   0.0        0.0
 46959.8       0.0  0.0           0.0        0.0   0.0        0.0
  1760.0       0.0  1.1623e5  53600.0        0.0   0.0        0.0
   400.0    2480.0  0.0       65524.5        0.0   0.0        0.0
 31200.0       0.0  0.0           0.0        0.0   7.75558e5  0.0
     0.0       0.0  0.0           0.0  …     0.0   0.0        0.0
     0.0       0.0  0.0           0.0        0.0   0.0        0.0
     0.0  116000.0  0.0           0.0        0.0   0.0        0.0
     0.0     867.2  0.0           0.0        0.0   0.0        0.0
     0.0       0.0  0.0           0.0        0.0   0.0        0.0
     0.0       0.0  0.0           0.0  …  3102.23  0.0        0.0
     0.0       0.0  0.0           0.0        0.0   0.0        0.0
     0.0       0.0  0.0           0.0        0.0   0.0        0.0
     ⋮                                 ⋱                      
     0.0       0.0  0.0           0.0        0.0   0.0  

In [646]:
new_cut, z_hat = add_cuts(x_hat)

G = new_cut["gradient"]
g = new_cut["intersection"]

Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26


2.1610772922702786e6

In [647]:
G

30×15 Matrix{Float64}:
  -1.94499    -37.164    -2.59542  …   -30.1117  -0.162214    -6.87191
  -2.15425    -34.36     -2.4279       -26.36    -0.143261    -6.57235
  -1.82609    -66.9999   -4.6173       -54.2792  -0.420597   -10.6968
  -0.621394   -65.2316   -4.74582      -52.6054  -0.919166   -10.2378
 -15.0245    -243.35    -17.2811      -186.701   -0.994974   -41.4555
  -1.39305    -15.6564   -1.2745   …   -12.155   -0.067607    -3.53728
  -3.23688    -50.5006   -3.50632      -40.6007  -0.593813    -8.74541
  -3.38401    -41.5205   -2.73216      -40.4691  -0.619772    -1.8679
  -4.08978    -52.4061   -4.33129      -41.4634  -0.326423    -8.49021
  -2.73119    -25.3339   -2.52129      -20.2484  -0.19625     -4.40718
  -1.84235    -18.1365   -1.6881   …   -13.8662  -0.133916    -3.96413
  -1.62243    -15.8673   -1.38041      -12.0747  -0.0645703   -3.68888
  -1.72657    -27.2297   -1.94048      -20.5954  -0.108758    -5.63564
   ⋮                               ⋱                      

In [648]:
g

2.1610772922702786e6

In [649]:
sum(x_hat .* G) + g

-1.7675175975429416e7

In [650]:
cuts

Any[]

In [651]:
model = Model(Gurobi.Optimizer)
# set_silent(model)
@variable(model, x[1:ncty, 1:nproc] >= 0)
@variable(model, theta >= M)

for k in 1:nproc
    for i in 1:ncty
        @constraint(model, x[i,k] <= 0.8*capacity[!, 2:end][i,k])
    end
end

x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];


###########
proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
pro_sink = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
    end
end
proS = (x.*pro_sink)*ones(nproc,1) 


@constraint(model, sum(x .* G) + g >= 0); 


@objective(model, Min, sum(proD - proS) + theta)
JuMP.optimize!(model);

Academic license - for non-commercial use only - expires 2024-12-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 465 rows, 451 columns and 1740 nonzeros
Model fingerprint: 0x89c09483
Coefficient statistics:
  Matrix range     [4e-03, 2e+02]
  Objective range  [3e-03, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 8e+10]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 450 rows and 277 columns
Presolve time: 0.00s
Presolved: 15 rows, 174 columns, 443 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -4.7938816e+06   8.693978e+05   0.000000e+00      0s
      17   -4.3702460e+05   0.000000e+00   0.000000e+00      0s

Solved in 17 iterations and 0.00 seconds
Optimal objective -4.370245978e+05

User-callback calls 50, time in user-callback 0.00 sec


In [652]:
x_hat = JuMP.value.(x)
x_hat22 = x_hat

30×15 Matrix{Float64}:
    0.0      0.0       0.0      0.0  …    0.0      0.0        0.0      0.0
    0.0      0.0       0.0      0.0       0.0      0.0        0.0      0.0
    0.0      0.0   11879.4  12175.2       0.0      0.0        0.0      0.0
    0.0   2480.0       0.0      0.0       0.0      0.0        0.0      0.0
 8716.07     0.0       0.0      0.0     396.332    0.0    79266.4      0.0
    0.0      0.0       0.0      0.0  …    0.0      0.0        0.0      0.0
    0.0      0.0       0.0      0.0       0.0      0.0        0.0      0.0
    0.0   9717.95      0.0      0.0       0.0      0.0        0.0      0.0
    0.0      0.0       0.0      0.0       0.0      0.0        0.0      0.0
    0.0      0.0       0.0      0.0       0.0      0.0        0.0      0.0
    0.0      0.0       0.0      0.0  …    0.0    317.065      0.0      0.0
    0.0      0.0       0.0      0.0       0.0      0.0        0.0      0.0
    0.0      0.0       0.0      0.0       0.0      0.0        0.0      0.0
  

In [654]:
new_cut, z_hat = add_cuts(x_hat22)

G = new_cut["gradient"]
g = new_cut["intersection"]

Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26


4.0435714285714286e6

In [655]:
z_hat, z_ub

(3.36133623300073e6, 1.0e8)

In [656]:
if z_hat < z_ub
    x_opt = x_hat
    z_ub = z_hat
end

3.36133623300073e6

In [657]:
push!(cuts, new_cut)

1-element Vector{Any}:
 Dict{String, Any}("gradient" => [-1.7394757707304322 -37.163985063095545 … -0.16221358056485538 -9.793952928375441; -2.154250748215941 -34.360006328422564 … -0.14326094365592024 -9.091820987467585; … ; -1.502431194837882 -14.628160558779728 … -0.060238688789963586 -6.093503531203845; -1.3319927442737913 -22.754249291784703 … -0.09715297450424931 -8.589546742209633], "intersection" => 4.0435714285714286e6)

In [660]:
res1 = masterprob(cuts)

Academic license - for non-commercial use only - expires 2024-12-26


Dict{String, Any} with 2 entries:
  "x_hat" => [4960.0 0.0 … 0.0 0.0; 46959.8 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.…
  "z_lb"  => -3.38704e6

In [663]:
z_lb = res1["z_lb"]
x_hat = res1["x_hat"]

30×15 Matrix{Float64}:
  4960.0       0.0  0.0           0.0  …     0.0   0.0        0.0
 46959.8       0.0  0.0           0.0        0.0   0.0        0.0
  1760.0       0.0  1.1623e5  53600.0        0.0   0.0        0.0
   400.0    2480.0  0.0       65524.5        0.0   0.0        0.0
 31200.0       0.0  0.0           0.0        0.0   7.75558e5  0.0
     0.0       0.0  0.0           0.0  …     0.0   0.0        0.0
     0.0       0.0  0.0           0.0        0.0   0.0        0.0
     0.0  116000.0  0.0           0.0        0.0   0.0        0.0
     0.0     867.2  0.0           0.0        0.0   0.0        0.0
     0.0       0.0  0.0           0.0        0.0   0.0        0.0
     0.0       0.0  0.0           0.0  …  3102.23  0.0        0.0
     0.0       0.0  0.0           0.0        0.0   0.0        0.0
     0.0       0.0  0.0           0.0        0.0   0.0        0.0
     ⋮                                 ⋱                      
     0.0       0.0  0.0           0.0        0.0   0.0  

In [664]:
xhat33 = x_hat;

In [667]:
(z_ub - z_lb) / min(abs(z_ub), abs(z_lb))

2.007647212906012

In [665]:
new_cut, z_hat = add_cuts(x_hat)

Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26


(Dict{String, Any}("gradient" => [-1.9449855844527981 -37.163985063095545 … -0.16221358056485538 -6.871906544096869; -2.154250748215941 -34.360006328422564 … -0.14326094365592024 -6.572346463264013; … ; -1.7367725050028822 -14.628160558779728 … -0.060238688789963586 -3.4168141557038445; -1.5236668691120065 -22.754249291784703 … -0.09715297450424931 -4.855920926459632], "intersection" => 2.1610772922702786e6), -599566.6527022659)

In [666]:
z_hat, z_ub

(-599566.6527022659, 3.36133623300073e6)

In [668]:
if z_hat < z_ub
    x_opt = x_hat
    z_ub = z_hat
end

-599566.6527022659

In [669]:
push!(cuts, new_cut)

2-element Vector{Any}:
 Dict{String, Any}("gradient" => [-1.7394757707304322 -37.163985063095545 … -0.16221358056485538 -9.793952928375441; -2.154250748215941 -34.360006328422564 … -0.14326094365592024 -9.091820987467585; … ; -1.502431194837882 -14.628160558779728 … -0.060238688789963586 -6.093503531203845; -1.3319927442737913 -22.754249291784703 … -0.09715297450424931 -8.589546742209633], "intersection" => 4.0435714285714286e6)
 Dict{String, Any}("gradient" => [-1.9449855844527981 -37.163985063095545 … -0.16221358056485538 -6.871906544096869; -2.154250748215941 -34.360006328422564 … -0.14326094365592024 -6.572346463264013; … ; -1.7367725050028822 -14.628160558779728 … -0.060238688789963586 -3.4168141557038445; -1.5236668691120065 -22.754249291784703 … -0.09715297450424931 -4.855920926459632], "intersection" => 2.1610772922702786e6)

In [674]:
cuts[1]["gradient"] == cuts[2]["gradient"]
cuts[1]["intersection"] == cuts[2]["intersection"]

false

In [676]:
res = masterprob(cuts)

Academic license - for non-commercial use only - expires 2024-12-26


Dict{String, Any} with 2 entries:
  "x_hat" => [4960.0 0.0 … 0.0 0.0; 46959.8 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.…
  "z_lb"  => -3.38704e6

In [681]:
res["x_hat"] - xhat11

30×15 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮

1.5957399771203836e6

In [420]:
if z_hat < z_ub
    x_opt = x_hat
    z_ub = z_hat
end

push!(cuts, new_cut)

1-element Vector{Any}:
 Dict{String, Any}("gradient" => [-0.20456365647918104 0.0 … 0.0 -1.53407435174625; 0.0 0.0 … 0.0 -1.322724125206875; … ; -0.9758627518664998 -0.16453433457900002 … 0.0 -1.4052619221375; -0.6028682999858039 0.0 … 0.0 -1.875], "intersection" => 3.5875e6)

In [421]:
res1 = masterprob(cuts)

Academic license - for non-commercial use only - expires 2024-12-26


Dict{String, Any} with 2 entries:
  "x_hat" => [4960.0 0.0 … 0.0 0.0; 48800.0 5600.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0…
  "z_lb"  => -1.42621e6

In [422]:
z_lb = res1["z_lb"]
x_hat = res1["x_hat"]

30×15 Matrix{Float64}:
  4960.0         0.0  0.0        0.0        …     0.0        0.0       0.0
 48800.0      5600.0  0.0        0.0              0.0        0.0       0.0
  1760.0         0.0  1.21684e5  0.0              0.0        0.0       0.0
   400.0      2480.0  0.0        1.24714e5        0.0    11948.6       0.0
 31200.0         0.0  0.0        0.0              0.0   800000.0       0.0
   961.301       0.0  0.0        0.0        …     0.0        0.0       0.0
     0.0         0.0  0.0        0.0              0.0        0.0       0.0
     0.0    116000.0  0.0        0.0              0.0        0.0       0.0
     0.0       867.2  0.0        0.0              0.0        0.0       0.0
     0.0         0.0  0.0        0.0              0.0        0.0       0.0
     0.0         0.0  0.0        0.0        …  3247.79       0.0   11948.6
     0.0         0.0  0.0        0.0              0.0        0.0       0.0
     0.0         0.0  0.0        0.0              0.0        0.0       0.0
  

In [427]:
new_cut, z_hat = add_cuts(x_hat)

Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26


(Dict{String, Any}("gradient" => [-0.20456365647918104 -0.2456769403350001 … -0.011335630170000055 -1.53407435174625; 0.0 0.0 … 0.0 -1.322724125206875; … ; -0.9758627518664998 -0.9758627518664998 … -0.7415214417014998 -1.4052619221375; -0.6028682999858039 -0.6612103934474999 … 0.0 -1.875], "intersection" => 3.5875e6), 461498.0955369733)

In [428]:
if z_hat < z_ub
    x_opt = x_hat
    z_ub = z_hat
end

push!(cuts, new_cut)

2-element Vector{Any}:
 Dict{String, Any}("gradient" => [-0.20456365647918104 0.0 … 0.0 -1.53407435174625; 0.0 0.0 … 0.0 -1.322724125206875; … ; -0.9758627518664998 -0.16453433457900002 … 0.0 -1.4052619221375; -0.6028682999858039 0.0 … 0.0 -1.875], "intersection" => 3.5875e6)
 Dict{String, Any}("gradient" => [-0.20456365647918104 -0.2456769403350001 … -0.011335630170000055 -1.53407435174625; 0.0 0.0 … 0.0 -1.322724125206875; … ; -0.9758627518664998 -0.9758627518664998 … -0.7415214417014998 -1.4052619221375; -0.6028682999858039 -0.6612103934474999 … 0.0 -1.875], "intersection" => 3.5875e6)

In [438]:
G = new_cut["gradient"]
g = new_cut["intersection"]
sum(x_hat .* G) + g

1.9218392077057904e6

In [434]:
res1 = masterprob(cuts)

Academic license - for non-commercial use only - expires 2024-12-26


Dict{String, Any} with 2 entries:
  "x_hat" => [4960.0 0.0 … 0.0 0.0; 48800.0 5600.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0…
  "z_lb"  => -1.42496e6

In [575]:
x_hat = xhat22

sum(x_hat, dims=1)

1×15 Matrix{Float64}:
 89281.3  1.24947e5  1.21684e5  1.24714e5  …  3247.79  8.11949e5  8.11949e5

In [576]:
s = 2
scen_dmd[s]

380000

In [577]:
model = Model(Gurobi.Optimizer)
@variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
@variable(model, unmeet >= 0)

cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == Omega[i,s,k] * x_hat[i,k]) for k in 1:nproc for i in 1:ncty]
cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,5] * scaler[k]) for k in cathode for j in 1:ncty]
cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,11] * scaler[k]) for k in cell for j in 1:ncty]
cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,14] * scaler[k]) for k in noncell for j in 1:ncty]
cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,15] * scaler[k]) for k in battery for j in 1:ncty] 

cstr_fmk = [@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) + unmeet == scen_dmd[s])]


transD = Vector{AffExpr}(undef, ncty)
for j in 1:ncty
    arc_emi = 0
    for i in 1:ncty
        amount = sum(y[i,j,k] for k in 1:nproc)
        arc_emi += amount * distance[!, 2:end][i,j] * EF_trans
    end
    transD[j] = arc_emi  # ncty*1 matrix
end


sink_ratio = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        sink_ratio[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
    end
end
overprod_penalty = sum((x_hat - sum(y, dims=2)[:,1,:]) .* sink_ratio)


############### Obj function ##############
@objective(model, Min, sum(transD) + unmeet * penalty + overprod_penalty)
JuMP.optimize!(model)

Academic license - for non-commercial use only - expires 2024-12-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 871 rows, 13501 columns and 26131 nonzeros
Model fingerprint: 0x41928716
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-05, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 8e+05]
Presolve removed 871 rows and 13501 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7463936e+06   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  1.746393644e+06

User-callback calls 35, time in user-callback 0.00 sec


In [578]:
qy_hat = JuMP.objective_value(model)
sub_y = JuMP.value.(y)
sub_unmeet = JuMP.value.(unmeet)

op_pi = [getdual(con) for con in cstr_op]
fmk_alp = [getdual(con) for con in cstr_fmk]

1-element Vector{Float64}:
 -0.22707614205000004

In [564]:
sub_unmeet

0.0

In [580]:
fmk_alp

1-element Vector{Float64}:
 -0.22707614205000004

In [581]:
op_pi

450-element Vector{Float64}:
  -2.217852577728768
  -1.8096119198642555
  -4.139828367142872
  -4.3497555574795825
 -12.959533437347783
  -1.5453125510964865
  -3.2368799747355994
  -3.3840086268748246
  -4.089782046186096
  -2.731193903281465
  -2.4000331986767733
  -1.622427094909849
  -1.7265650878339138
   ⋮
  -6.524582569571739
  -6.96955917211077
  -6.213796600566572
  -3.4625074102348945
  -2.9901558073654395
  -2.8659347879679085
  -2.8705975812700872
  -2.8164748005715863
  -2.383664967976352
  -3.273113366586142
  -2.346138405503845
  -3.3624706001596314

In [574]:
fmk_alp

1-element Vector{Float64}:
 -0.22707614205000004

In [573]:
op_pi

450-element Vector{Float64}:
  -2.217852577728768
  -1.8096119198642555
  -4.139828367142872
  -4.3497555574795825
 -12.959533437347783
  -1.5453125510964865
  -3.2368799747355994
  -3.3840086268748246
  -4.089782046186096
  -2.731193903281465
  -2.4000331986767733
  -1.622427094909849
  -1.7265650878339138
   ⋮
  -6.524582569571739
  -6.96955917211077
  -6.213796600566572
  -3.4625074102348945
  -2.9901558073654395
  -2.8659347879679085
  -2.8705975812700872
  -2.8164748005715863
  -2.383664967976352
  -3.273113366586142
  -2.346138405503845
  -3.3624706001596314

----

In [630]:
scen_eff = CSV.File(string(path,"Scenarios/Scen_Efficiency2.csv"), header=1, delim=",") |> DataFrame
scen_eff = Matrix(scen_eff)[:, 2:end]
num_omega = size(scen_eff)[2]

scen_dmd = CSV.File(string(path,"Scenarios/Scen_Demand.csv"), header=1, delim=",") |> DataFrame
scen_dmd = Matrix(scen_dmd)[2:end]
# scen_dmd = Matrix(scen_dmd)[2:end][list]
# scen_dmd = ones(num_omega) .* 452000

Omega = ones(Float64, ncty, num_omega, nproc);  # Omega[ncty,nscena,nproc]
Omega[:, :, 1] = scen_eff;

In [631]:
function sub_test(s,x)
    model = Model(Gurobi.Optimizer)
    set_silent(model)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    @variable(model, unmeet >= 0)

    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == Omega[i,s,k] * x[i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x[j,5] * scaler[k]) for k in cathode for j in 1:ncty]
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x[j,11] * scaler[k]) for k in cell for j in 1:ncty]
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x[j,14] * scaler[k]) for k in noncell for j in 1:ncty]
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x[j,15] * scaler[k]) for k in battery for j in 1:ncty] 

    cstr_fmk = [@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) + unmeet >= scen_dmd[s])]


    transD = Vector{AffExpr}(undef, ncty)
    for j in 1:ncty
        arc_emi = 0
        for i in 1:ncty
            amount = sum(y[i,j,k] for k in 1:nproc)
            arc_emi += amount * distance[!, 2:end][i,j] * EF_trans
        end
        transD[j] = arc_emi  # ncty*1 matrix
    end


    pro_sink = zeros(AffExpr, ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = (sum(y[i,j,k] for j in 1:ncty)) * regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = sum(pro_sink)

    @objective(model, Min, sum(transD) + unmeet*penalty)
    JuMP.optimize!(model)
    
    op_pi = [getdual(con) for con in cstr_op]
    cth_pi = [getdual(con) for con in cstr_cth]
    cell_pi = [getdual(con) for con in cstr_cell]
    noncell_pi = [getdual(con) for con in cstr_noncell]
    battery_pi = [getdual(con) for con in cstr_battery]
    fmk_alp = [getdual(con) for con in cstr_fmk]
    
    sol_unmeet = JuMP.value.(unmeet)
    sol_y = JuMP.value.(y)
    
    res = Dict(["y"=>sol_y, 
            "unmeet"=>sol_unmeet, 
            "dual_op"=>op_pi, 
            "dual_cth"=>cth_pi,
            "dual_cell"=>cell_pi,
            "dual_ncell"=>noncell_pi,
            "dual_bty"=>battery_pi,
            "dual_alp"=>fmk_alp])

    return res
    
end


sub_test (generic function with 2 methods)

In [127]:
x = xhat1;

In [129]:
lsop = []
lscth = []
lscell = []
lsncell = []
lsbty = []
lsalp = []
lsunmeet = []

for s in 1: num_omega
    res = sub_test(s,x)
    push!(lsop, res["dual_op"])
    push!(lscth, res["dual_cth"])
    push!(lscell, res["dual_cell"])
    push!(lsncell, res["dual_ncell"])
    push!(lsbty, res["dual_bty"])
    push!(lsalp, res["dual_alp"])
    push!(lsunmeet, res["unmeet"])
end
    
    

Academic license - for non-commercial use only - expires 2024-12-26


In [130]:
op1 = sum(lsop)/16

450-element Vector{Float64}:
 -0.1232609773371105
 -0.11310074499151597
 -0.22513408098039828
 -0.22115169626200515
 -0.795324507948924
 -0.05181723322721791
 -0.16903349340150622
 -0.15902050501358278
 -0.17483990659575604
 -0.08475767109770407
 -0.05899656267479836
 -0.05182618582164681
 -0.08872394150524462
  ⋮
 -0.6761852475267337
 -0.7117408715031106
 -0.6211996211116608
 -0.48172756152358714
 -0.4471660134736525
 -0.4174731006658068
 -0.3980953570012555
 -0.42433941588759916
 -0.397553144616022
 -0.4602591300678214
 -0.38084397070024034
 -0.5368466713881019

In [131]:
x = xhat2;

In [132]:
lsop = []
lscth = []
lscell = []
lsncell = []
lsbty = []
lsalp = []
lsunmeet = []

for s in 1: num_omega
    res = sub_test(s,x)
    push!(lsop, res["dual_op"])
    push!(lscth, res["dual_cth"])
    push!(lscell, res["dual_cell"])
    push!(lsncell, res["dual_ncell"])
    push!(lsbty, res["dual_bty"])
    push!(lsalp, res["dual_alp"])
    push!(lsunmeet, res["unmeet"])
end

Academic license - for non-commercial use only - expires 2024-12-26


In [133]:
op2 = sum(lsop)/16

450-element Vector{Float64}:
 -0.12396945422273549
 -0.11310074499151597
 -0.244092941061117
 -0.2572133904571614
 -0.795324507948924
 -0.08193570255821789
 -0.18765866653566243
 -0.196854207294364
 -0.24096504600131852
 -0.15605328706977906
 -0.13535574303198583
 -0.08675536154655304
 -0.0932639861043071
  ⋮
 -0.6761852475267337
 -0.7117408715031106
 -0.6211996211116608
 -0.48172756152358714
 -0.4471660134736525
 -0.4174731006658068
 -0.3980953570012555
 -0.42433941588759916
 -0.397553144616022
 -0.4602591300678214
 -0.38084397070024034
 -0.5368466713881019

In [134]:
op1 - op2

450-element Vector{Float64}:
 0.0007084768856249896
 0.0
 0.01895886008071873
 0.03606169419515626
 0.0
 0.03011846933099998
 0.01862517313415621
 0.03783370228078123
 0.06612513940556247
 0.07129561597207498
 0.07635918035718747
 0.03492917572490623
 0.004540044599062473
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [121]:
s = 1

1

In [123]:
model = Model(Gurobi.Optimizer)
@variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
@variable(model, unmeet >= 0)

cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == Omega[i,s,k] * x_hat[i,k]) for k in 1:nproc for i in 1:ncty]
cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,5] * scaler[k]) for k in cathode for j in 1:ncty]
cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,11] * scaler[k]) for k in cell for j in 1:ncty]
cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,14] * scaler[k]) for k in noncell for j in 1:ncty]
cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,15] * scaler[k]) for k in battery for j in 1:ncty] 

cstr_fmk = [@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) + unmeet >= scen_dmd[s])]


transD = Vector{AffExpr}(undef, ncty)
for j in 1:ncty
    arc_emi = 0
    for i in 1:ncty
        amount = sum(y[i,j,k] for k in 1:nproc)
        arc_emi += amount * distance[!, 2:end][i,j] * EF_trans
    end
    transD[j] = arc_emi  # ncty*1 matrix
end


pro_sink = zeros(AffExpr, ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        pro_sink[i,k] = (sum(y[i,j,k] for j in 1:ncty)) * regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
    end
end
proS = sum(pro_sink)

@objective(model, Min, sum(transD) + unmeet*penalty - proS)
JuMP.optimize!(model)

Academic license - for non-commercial use only - expires 2024-12-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 871 rows, 13501 columns and 26131 nonzeros
Model fingerprint: 0xe3b3a2fd
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-05, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 5e+05]
Presolve removed 871 rows and 13501 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.2108645e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  6.210864452e+04

User-callback calls 30, time in user-callback 0.00 sec


In [124]:
qy_hat = JuMP.objective_value(model)
sub_y = JuMP.value.(y)
sub_unmeet = JuMP.value.(unmeet)

375764.1741144483

In [125]:
[getdual(con) for con in cstr_fmk]

1-element Vector{Float64}:
 5.0

In [ ]:

    
    ################## get dual and calculate cuts ################
    dual_pi = zeros(ncty,nproc)

    op_pi = [getdual(con) for con in cstr_op]
    cth_pi = [getdual(con) for con in cstr_cth]
    cell_pi = [getdual(con) for con in cstr_cell]
    noncell_pi = [getdual(con) for con in cstr_noncell]
    battery_pi = [getdual(con) for con in cstr_battery]
    fmk_alp = [getdual(con) for con in cstr_fmk]


    dual_pi[:, cathode] = reshape(cth_pi, ncty, length(cathode)) .* Omega[:,s,cathode] .* repeat(scaler[cathode]', ncty, 1)
    dual_pi[:, cell] = reshape(cell_pi, ncty, length(cell)) .* Omega[:,s,cell] .* repeat(scaler[cell]', ncty, 1)
    dual_pi[:, noncell] = reshape(noncell_pi, ncty, length(noncell)) .* Omega[:,s,noncell] .* repeat(scaler[noncell]', ncty, 1)
    dual_pi[:, battery] = reshape(battery_pi, ncty, length(battery)) .* Omega[:,s,battery] .* repeat(scaler[battery]', ncty, 1)
    dual_pi += reshape(op_pi, ncty, nproc) .* Omega[:,s,:]
    
    dual_alpha = sum(fmk_alp) * scen_dmd[s]

---

In [632]:
scen_eff = CSV.File(string(path,"Scenarios/Scen_Efficiency2.csv"), header=1, delim=",") |> DataFrame
scen_eff = Matrix(scen_eff)[:, 2:end]
num_omega = size(scen_eff)[2]

scen_dmd = CSV.File(string(path,"Scenarios/Scen_Demand.csv"), header=1, delim=",") |> DataFrame
scen_dmd = Matrix(scen_dmd)[2:end]
# scen_dmd = Matrix(scen_dmd)[2:end][list]
# scen_dmd = ones(num_omega) .* 452000

Omega = ones(Float64, ncty, num_omega, nproc);  # Omega[ncty,nscena,nproc]
Omega[:, :, 1] = scen_eff;

In [633]:
function gen_fea(x_hat)
    ls = []
    for s in 1: num_omega
        model = Model(Gurobi.Optimizer)
        set_silent(model)
        @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
        @variable(model, unmeet >= 0)

        cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == Omega[i,s,k] * x_hat[i,k]) for k in 1:nproc for i in 1:ncty]
        cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
        cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,11]*scaler[k]) for k in cell for j in 1:ncty]
        cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
        cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,15]*scaler[k]) for k in battery for j in 1:ncty] 

        cstr_fmk = [@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) + unmeet == scen_dmd[s])]
        
        transD = Vector{AffExpr}(undef, ncty)
        for j in 1:ncty
            arc_emi = 0
            for i in 1:ncty
                amount = sum(y[i,j,k] for k in 1:nproc)
                arc_emi += amount * distance[!, 2:end][i,j] * EF_trans
            end
            transD[j] = arc_emi  # ncty*1 matrix
        end


        pro_sink = zeros(AffExpr, ncty, nproc)
        for k in 1:nproc
            for i in 1:ncty
                pro_sink[i,k] = (sum(y[i,j,k] for j in 1:ncty)) * regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
            end
        end
        proS = sum(pro_sink)
        
        
        sink_ratio = zeros(ncty, nproc)
        for k in 1:nproc
            for i in 1:ncty
                sink_ratio[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
            end
        end
        overprod_penalty = sum((x_hat - sum(y, dims=2)[:,1,:]) .* sink_ratio)
        

        @objective(model, Min, sum(transD) + unmeet * penalty + overprod_penalty)
        JuMP.optimize!(model)

        if termination_status(model) == MOI.OPTIMAL
            push!(ls, s)
        end
    end
    
    return ls
    
end

gen_fea (generic function with 1 method)

In [635]:
ls = gen_fea(x_hat)

Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only -

9-element Vector{Any}:
  1
  4
  5
 11
 21
 31
 37
 41
 49